<a href="https://colab.research.google.com/github/pavankumar1909/AI-ML-Demo-project/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow opencv-python numpy

**import libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

**Choose Model**

In [ ]:
# Choose model: "resnet" or "efficientnet"
model_choice = "resnet"

**Load Data**

In [ ]:
import zipfile

# Update the path if the file is located elsewhere
file_path = "/content/Trash_ClassificationLarge.zip"  # Replace with the correct path

try:
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall("trash_images")
    print(f"File '{file_path}' extracted successfully!")
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found. Please check the path and ensure the file exists.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


File '/content/Trash_ClassificationLarge.zip' extracted successfully!


In [ ]:

data_dir = '/content/trash_images/Trash_Classification'



# --- Load your dataset using OpenCV ---
def load_data(data_dir, img_size=(224, 224)):
    images, labels = [], []
    class_names = os.listdir(data_dir)
    for label_idx, class_name in enumerate(class_names):
        class_path = os.path.join(data_dir, class_name)
        for file in os.listdir(class_path):
            img_path = os.path.join(class_path, file)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size)
                images.append(img)
                labels.append(label_idx)
    return np.array(images), np.array(labels), class_names

In [ ]:
X, y, class_names = load_data(data_dir)
print(class_names)
print(f"Loaded {len(X)} images in {len(class_names)} classes.")

['plastic', 'paper', ' noobject', 'glass', 'metal']
Loaded 3254 images in 5 classes.


**Split DataSet**

In [ ]:
# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

**Normalise and Preprocess**

In [ ]:
# Normalize and preprocess
if model_choice == "resnet":
    X_train = resnet_preprocess(X_train.astype('float32'))
    X_val = resnet_preprocess(X_val.astype('float32'))
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
elif model_choice == "efficientnet":
    X_train = efficientnet_preprocess(X_train.astype('float32'))
    X_val = efficientnet_preprocess(X_val.astype('float32'))
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Build Models**

In [ ]:
# Build model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(class_names), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

**Freeze Base Model Layer**

In [ ]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,850,629 (90.98 MB)

 Trainable params: 262,917 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

**Train**

In [ ]:
# Train
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)

Epoch 1/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 644s 8s/step - accuracy: 0.6514 - loss: 0.9942 - val_accuracy: 0.8249 - val_loss: 0.4834
Epoch 2/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 643s 8s/step - accuracy: 0.9217 - loss: 0.2325 - val_accuracy: 0.8525 - val_loss: 0.4134
Epoch 3/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 623s 8s/step - accuracy: 0.9604 - loss: 0.1245 - val_accuracy: 0.8233 - val_loss: 0.5951
Epoch 4/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 698s 8s/step - accuracy: 0.9792 - loss: 0.0854 - val_accuracy: 0.8725 - val_loss: 0.4058
Epoch 5/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 689s 8s/step - accuracy: 0.9929 - loss: 0.0332 - val_accuracy: 0.8725 - val_loss: 0.4104


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from PIL import Image
from io import BytesIO

# Function to capture photo
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = '📸 Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            div.remove();

            return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Capture photo
image_path = take_photo()


<IPython.core.display.Javascript object>

**TEST**

In [ ]:
# Test on a new image
def predict_image(image_path):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (224, 224))
    img_preprocessed = resnet_preprocess(np.expand_dims(img_resized.astype('float32'), axis=0)) if model_choice == "resnet" else efficientnet_preprocess(np.expand_dims(img_resized.astype('float32'), axis=0))
    prediction = model.predict(img_preprocessed)
    predicted_class = class_names[np.argmax(prediction)]
    print(f"Predicted: {predicted_class}")

print(f"paper:{predict_image('/content/photo.jpg')}")

# print(f"paper:{predict_image('/content/cardboard.jpeg')}")
# print(f"paper:{predict_image('/content/crushed_newspaper.jpeg')}")
# print(f"paper:{predict_image('/content/crushed_paper.jpeg')}")
# print(f"plastic:{predict_image('/content/garbage_bag.png')}")
# print(f"glass:{predict_image('/content/glassBottle.png')}")
# print(f"glass:{predict_image('/content/glass_jars.jpeg')}")
# print(f"glass:{predict_image('/content/glass_toys.jpeg')}")
# print(f"paper:{predict_image('/content/multiple_cardboards.jpeg')}")
# print(f"paper:{predict_image('/content/multiple_paper.jpeg')}")
# print(f"plastic:{predict_image('/content/plastic_bottle.png')}")
# print(f"plastic:{predict_image('/content/plastic_cover.png')}")
# print(f"plastic:{predict_image('/content/plastic_pipes.jpeg')}")
# print(f"paper:{predict_image('/content/ruled_paper.jpeg')}")
# print(f"noitem:{predict_image('/content/whitebg.webp')}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted:  noobject
paper:None


**save trained model to local machine**

In [ ]:
model.save("Classification_img2.h5")
#from google.colab import files
#files.download("Image_Classification.h5")

In [ ]:
from google.colab import files
files.download("Classification_img2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>